#### **Взять интеграл с заданной точностью eps:**

In [69]:
import numpy as np
import pandas as pd
from random import random

Запишем условие:

In [70]:
def f(x):
    return np.cos(x)/(x + 2)

Метод трапеций:

In [71]:
def trapezoid(n, f=f, a=0.4, b=1.2):
    h = (b - a) / n
    return sum([h * (f(i) + f(i + h)) / 2 for i in np.arange(a, b - h, h)])

Используя метод Симпсона, заменим подынтегральную функцию параболой:

In [72]:
def simpsons_method(N, a=0.4, b=1.2):
    h = (b - a) / N
    return sum([h / 6 * (f(i - h / 2) + 4 * f(i) + f(i + h / 2)) for i in np.arange(a + h / 2, b - h / 2, h)])


Создадим таблицу для проверки сходимости:

In [73]:
data = pd.DataFrame(np.array([[abs(trapezoid(i) - trapezoid(i * 2)) for i in [2 ** j for j in range(1, 9)]],
                            [np.log2(abs(trapezoid(i * 2) - trapezoid(i * 4)) / abs(trapezoid(i) - trapezoid(i * 2))) for i in [2 ** j for j in range(1, 9)]], 
                            [abs(simpsons_method(i) - simpsons_method(i * 2)) for i in [2 ** j for j in range(1, 9)]], 
                            [np.log2(abs(simpsons_method(i * 2) - simpsons_method(i * 4)) / abs(simpsons_method(i) - simpsons_method(i * 2))) for i in [2 ** j for j in range(1, 9)]]]).T, 
                    columns=['eps_tr', 'P_tr', 'eps_simp', 'P_simp'], 
                    index=[2 ** i for i in range(1, 9)])
data

,eps_tr,P_tr,eps_simp,P_simp
2,0.029292,-0.839741,0.029287,-0.843929
4,0.016367,-1.244476,0.016316,-0.330999
8,0.006908,-0.185681,0.012971,-2.144750
16,0.006074,-2.075630,0.002933,-0.975028
32,0.001441,-0.986987,0.001492,-1.037643
64,0.000727,-1.019220,0.000727,-0.025594
128,0.000359,-0.012939,0.000714,-2.009706
256,0.000355,-2.004888,0.000177,-0.998379


ЧТО БУДЕТ, ЕСЛИ В ДАННЫЕ ВНЕСТИ ШУМ?

In [74]:
def trapezoid_noize(n, f=f, a=0.4, b=1.2):
    h = (b - a) / n
    return sum([h * (f(i) + f(i + h)) / 2 + random() for i in np.arange(a, b - h, h)])

def f_i_noize(i, N, f=f, a=0.4, b=1.2):
    return f(a + ((b - a) * i)/(2 * N))

def simpsons_method_noize(N, a=0.4, b=1.2, f_i=f_i_noize):
    return ((b - a)/(6 * N)) * (f_i(0, N) + 4 * sum([f_i(i, N) + random() for i in range(1, 2 * N)]) + 2 * sum([f_i(i, N) + random() for i in range(2, 2 * N, 2)]) + f_i(2 * N, N))

Посмотрим на сходимость теперь:

In [75]:
data = pd.DataFrame(np.array([[abs(trapezoid_noize(i) - trapezoid_noize(i * 2)) for i in [2 ** j for j in range(2, 6)]],
                            [np.log2(abs(trapezoid_noize(i * 2) - trapezoid_noize(i * 4)) / abs(trapezoid_noize(i) - trapezoid_noize(i * 2))) for i in [2 ** j for j in range(2, 6)]], 
                            [abs(simpsons_method_noize(i) - simpsons_method_noize(i * 2)) for i in [2 ** j for j in range(2, 6)]], 
                            [np.log2(abs(simpsons_method_noize(i * 2) - simpsons_method_noize(i * 4)) / abs(simpsons_method_noize(i) - simpsons_method_noize(i * 2))) for i in [2 ** j for j in range(2, 6)]]]).T, 
                    columns=['eps_tr', 'P_tr', 'eps_simp', 'P_simp'], 
                    index=[2 ** i for i in range(2, 6)])
data

,eps_tr,P_tr,eps_simp,P_simp
4,1.785254,2.112116,0.149484,-0.113416
8,4.857341,0.948886,0.075186,-1.247784
16,10.497556,0.920434,0.126235,-0.454258
32,12.529304,1.390669,0.009330,0.335923


Можем заметить, что метод Симпсона более устойчив к обработке шума.

#### **Вычислить производную:**

In [76]:
def f(x):
    return np.exp(x)

Найдём с помощью правосторонней и двусторонней разностей:

In [77]:
def right(N, x=1):
    h = 10 ** -N
    return (f(x) - f(x - h)) / h

def bilateral(N, x=1):
    h = 10 ** -N
    return (f(x + h) - f(x - h)) / (2 * h)

Сравним результаты работы методов с оригинальным числом e, который является значением производной при x = 1:

In [78]:
print(right(5))
print(bilateral(5))
print(np.e)

2.7182682370785467
2.718281828517632
2.718281828459045


Сразу видим, что двусторонний метод работает лучше.

Создадим таблицу для проверки сходимости:

In [79]:
data = pd.DataFrame(np.array([[abs(right(i) - right(i * 2)) for i in [j for j in range(1, 5)]],
                            [np.log2(abs(right(i * 2) - right(i * 4)) / abs(right(i) - right(i * 2))) for i in [j for j in range(1, 5)]], 
                            [abs(bilateral(i) - bilateral(i * 2)) for i in [j for j in range(1, 5)]], 
                            [np.log2(abs(bilateral(i * 2) - bilateral(i * 4)) / abs(bilateral(i) - bilateral(i * 2))) for i in [j for j in range(1, 5)]]]).T, 
                    columns=['eps_r', 'P_r', 'eps_bi', 'P_bi'], 
                    index=[i for i in range(1, 5)])
data

,eps_r,P_r,eps_bi,P_bi
1,0.117948,-3.136742,4.487431e-03,-6.630222
2,0.013410,-6.624622,4.530039e-05,-11.990424
3,0.001357,-7.025871,4.532101e-07,8.857846
4,0.000136,14.287831,1.113332e-08,27.863236


Как можно увидеть, при N->+inf и, соответственно, при h->0, погрешность eps->0.

ЧТО БУДЕТ, ЕСЛИ В ДАННЫЕ ВНЕСТИ ШУМ?

In [80]:
def right_noize(N, x=1):
    h = 10 ** -N
    return (f(x + random()) - f(x - h + random())) / h

def bilateral_noize(N, x=1):
    h = 10 ** -N
    return (f(x + h + random()) - f(x - h + random())) / (2 * h)

Посмотрим на сходимость теперь:

In [81]:
data = pd.DataFrame(np.array([[abs(right_noize(i) - right_noize(i * 2)) for i in [j for j in range(1, 5)]],
                            [np.log2(abs(right_noize(i * 2) - right_noize(i * 4)) / abs(right_noize(i) - right_noize(i * 2))) for i in [j for j in range(1, 5)]], 
                            [abs(bilateral_noize(i) - bilateral_noize(i * 2)) for i in [j for j in range(1, 5)]], 
                            [np.log2(abs(bilateral_noize(i * 2) - bilateral_noize(i * 4)) / abs(bilateral_noize(i) - bilateral_noize(i * 2))) for i in [j for j in range(1, 5)]]]).T, 
                    columns=['eps_r', 'P_r', 'eps_bi', 'P_bi'], 
                    index=[i for i in range(1, 5)])
data

,eps_r,P_r,eps_bi,P_bi
1,5.475577e+01,10.549697,2.164940e+02,6.873737
2,4.867450e+03,11.135876,1.266936e+04,13.989650
3,1.531773e+06,19.530445,7.400171e+05,20.967178
4,6.696157e+07,25.929278,6.109408e+07,24.867975


Значения расходятся.